In [1]:
import os
import sys
from osgeo import gdal, ogr
from math import ceil
import geopandas
import pandas as pd
from pyproj import Proj, transform
import numpy as np
from shapely.geometry import Point
#from IPython.core.debugger import set_trace
##python evaluation.py dank.shp -11724369.15 -11644519.67 4810066.59 4852046.60 100 100
def create_grid(outputGridfn, xmin, xmax, ymin, ymax, gridHeight, gridWidth):

    # convert sys.argv to float
    xmin = float(xmin)
    xmax = float(xmax)
    ymin = float(ymin)
    ymax = float(ymax)
    gridWidth = float(gridWidth)
    gridHeight = float(gridHeight)

    # get rows
    rows = ceil((ymax - ymin) / gridHeight)
    # get columns
    cols = ceil((xmax - xmin) / gridWidth)

    # start grid cell envelope
    ringXleftOrigin = xmin
    ringXrightOrigin = xmin + gridWidth
    ringYtopOrigin = ymax
    ringYbottomOrigin = ymax - gridHeight

    # create output file
    outDriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputGridfn):
        os.remove(outputGridfn)
    outDataSource = outDriver.CreateDataSource(outputGridfn)
    outLayer = outDataSource.CreateLayer(
        outputGridfn, geom_type=ogr.wkbPolygon)
    featureDefn = outLayer.GetLayerDefn()

    # create grid cells
    countcols = 0
    while countcols < cols:
        countcols += 1

        # reset envelope for rows
        ringYtop = ringYtopOrigin
        ringYbottom = ringYbottomOrigin
        countrows = 0

        while countrows < rows:
            countrows += 1
            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYtop)
            ring.AddPoint(ringXrightOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYbottom)
            ring.AddPoint(ringXleftOrigin, ringYtop)
            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)

            # add new geom to layer
            outFeature = ogr.Feature(featureDefn)
            outFeature.SetGeometry(poly)
            outLayer.CreateFeature(outFeature)
            outFeature.Destroy

            # new envelope for next poly
            ringYtop = ringYtop - gridHeight
            ringYbottom = ringYbottom - gridHeight

        # new envelope for next poly
        ringXleftOrigin = ringXleftOrigin + gridWidth
        ringXrightOrigin = ringXrightOrigin + gridWidth

    # Close DataSources
    outDataSource.Destroy()
    return outDataSource
grid = create_grid('dank.shp', -11724369.15, -11644519.67, 4810066.59, 4852046.60, 100, 100)

In [2]:
polys = geopandas.read_file('dank.shp')

In [3]:
crime_points = pd.read_csv('crime_denver_clean.csv')

In [4]:
data_crime = crime_points[['GEO_LON','GEO_LAT','REPORTED_DATE']]

In [5]:
inProj =  Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
points = np.vectorize(transform)(inProj,outProj,data_crime['GEO_LON'], data_crime['GEO_LAT'])

In [6]:
df = pd.DataFrame({'Longitude':points[0], 'Latitude':points[1]})
df['Coordinates'] = list(zip(df.Longitude, df.Latitude))
df['Coordinates'] = df['Coordinates'].apply(Point)
points = geopandas.GeoDataFrame(df, geometry='Coordinates')


In [36]:
points

,Latitude,Longitude,Coordinates
index_left,,,
0,4.833037e+06,-1.166738e+07,POINT (-11667382.59415555 4833037.111823073)
1,4.834842e+06,-1.166422e+07,POINT (-11664215.88860095 4834842.063949543)
2,4.817161e+06,-1.168380e+07,POINT (-11683802.21904756 4817161.328104024)
3,4.822833e+06,-1.168203e+07,POINT (-11682027.66391287 4822832.736933908)
4,4.824918e+06,-1.168358e+07,POINT (-11683578.37781547 4824917.85437972)
5,4.828687e+06,-1.168431e+07,POINT (-11684308.41103609 4828687.321722209)
6,4.827188e+06,-1.169139e+07,POINT (-11691389.93365121 4827187.747962995)
7,4.825832e+06,-1.168674e+07,POINT (-11686742.52302178 4825831.958892724)
8,4.827777e+06,-1.168762e+07,POINT (-11687616.98214975 4827777.269492989)


In [59]:
pointInPoly = geopandas.sjoin(points[['Coordinates']],polys[['geometry']] , op='within') 

In [60]:
pointInPoly

,Coordinates,index_right
0,POINT (-11667382.59415555 4833037.111823073),239170
6814,POINT (-11667382.59415555 4833037.111823073),239170
7667,POINT (-11667382.59415555 4833037.111823073),239170
9137,POINT (-11667382.59415555 4833037.111823073),239170
10909,POINT (-11667382.59415555 4833037.111823073),239170
12775,POINT (-11667382.59415555 4833037.111823073),239170
20082,POINT (-11667382.59415555 4833037.111823073),239170
20498,POINT (-11667382.59415555 4833037.111823073),239170
46784,POINT (-11667382.59415555 4833037.111823073),239170
47076,POINT (-11667382.59415555 4833037.111823073),239170


In [61]:
counts = pointInPoly.groupby('index_right').size()

In [62]:
counts

index_right
265         1
9933        2
41827       6
54106     460
55788       1
99091      15
99094       2
99097       7
99510       6
99511      19
99516       1
99520       1
99930       1
99931       7
99933       3
99935       2
99937       4
99938       2
99941       2
100351     14
100352      2
100356      1
100357      1
100358      1
100363      1
100771      2
100772      4
100773      1
100774      4
100775      2
         ... 
305864      1
305871      1
305872      1
306269      5
306275      1
306276      2
306280      1
306282      1
306295      2
306678      8
306679     46
306681     14
306704      8
306706      6
306717     15
306720     18
307086      1
307094      1
307112      1
310497      1
312480      1
315907      1
318845      1
321811      1
322210      1
327266      1
327292      2
328489      1
332668      7
335188      1
Length: 32559, dtype: int64

In [66]:
polys

,FID,geometry
index_right,,
0,0,"POLYGON ((-11724369.15 4852046.6, -11724269.15..."
1,1,"POLYGON ((-11724369.15 4851946.6, -11724269.15..."
2,2,"POLYGON ((-11724369.15 4851846.6, -11724269.15..."
3,3,"POLYGON ((-11724369.15 4851746.6, -11724269.15..."
4,4,"POLYGON ((-11724369.15 4851646.6, -11724269.15..."
5,5,"POLYGON ((-11724369.15 4851546.6, -11724269.15..."
6,6,"POLYGON ((-11724369.15 4851446.6, -11724269.15..."
7,7,"POLYGON ((-11724369.15 4851346.6, -11724269.15..."
8,8,"POLYGON ((-11724369.15 4851246.6, -11724269.15..."


In [ ]:
32559